In [1]:
import pandas as pd
import numpy as np
import sys
import os

# Adjust the path to point to external/AlphaPEM
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.sampling.sampler import get_polarisation_curve_samples, build_fixed_parameters, sample_parameters, PARAMETER_RANGES

In [2]:
data = pd.read_pickle('../data/raw/N10_levels4_seed42.pkl')

In [4]:
exploaded_df = data.explode(['ifc', 'Ucell'])

In [5]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np
from pysr import PySRRegressor

vars = list(PARAMETER_RANGES.keys())
vars.append('ifc')

X = exploaded_df[vars].values
y = exploaded_df['Ucell'].values

kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_scores = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = PySRRegressor(
        niterations=100,
        binary_operators=["+", "-", "*", "/"],
        unary_operators=["exp", "log", "sqrt", "square", "cube"],
        loss="loss(x, y) = (x - y)^2",
        model_selection="best",
        maxsize=20,
        maxdepth=5,
        ncyclesperiteration=30,
        verbosity=1,
    )
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)

print(f"Average MSE: {np.mean(mse_scores):.4f} ± {np.std(mse_scores):.4f}")


Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


c:\Users\dejvi\Documents\GitHub\Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\.venv\Lib\site-packages\pysr\sr.py:1017: FutureWarning: `loss` has been renamed to `elementwise_loss` in PySRRegressor. Please use that instead.
  warnings.warn(
c:\Users\dejvi\Documents\GitHub\Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\.venv\Lib\site-packages\pysr\sr.py:1017: FutureWarning: `ncyclesperiteration` has been renamed to `ncycles_per_iteration` in PySRRegressor. Please use that instead.
  warnings.warn(
c:\Users\dejvi\Documents\GitHub\Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\.venv\Lib\site-packages\pysr\sr.py:2776: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
Compiling Julia backend...
[ Info: Started!



Expressions evaluated per second: 1.320e+04
Progress: 167 / 3100 total iterations (5.387%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           5.893e+08  1.594e+01  y = 2352.2
3           5.854e+08  3.378e-03  y = x₃ / 0.00097575
4           5.514e+08  5.984e-02  y = sqrt(3.435 / x₁₇)
7           4.407e+08  7.470e-02  y = 4.9854 / (log(x₅) + sqrt(x₁₇))
10          3.339e+08  9.250e-02  y = (cube(0.71062 - x₇) / x₁₇) * (x₅ + -0.29915)
18          2.903e+08  1.749e-02  y = ((x₁₆ - (x₈ - x₉)) * log(square(x₁₆))) / (log(x₃ - 0.8...
                                      8193) + (x₁₀ + -1))
───────────────────────────────────────────────────────────────────────────────────────────────────
════════════════════════════════════════════════════════════════════════════════════════════════════
Pr

[ Info: Final population:
[ Info: Results saved to:
c:\Users\dejvi\Documents\GitHub\Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\.venv\Lib\site-packages\pysr\sr.py:1017: FutureWarning: `loss` has been renamed to `elementwise_loss` in PySRRegressor. Please use that instead.
  warnings.warn(
c:\Users\dejvi\Documents\GitHub\Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\.venv\Lib\site-packages\pysr\sr.py:1017: FutureWarning: `ncyclesperiteration` has been renamed to `ncycles_per_iteration` in PySRRegressor. Please use that instead.
  warnings.warn(
c:\Users\dejvi\Documents\GitHub\Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\.venv\Lib\site-packages\pysr\sr.py:2776: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 1.370e+04
Progress: 188 / 3100 total iterations (6.065%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           5.899e+08  1.594e+01  y = 2250.3
3           5.878e+08  1.786e-03  y = 2226.4 / x₄
4           5.824e+08  9.319e-03  y = cube(exp(exp(x₅)))
5           5.614e+08  3.662e-02  y = (x₁₂ / x₁₆) / 0.033262
9           5.571e+08  1.928e-03  y = x₆ * ((-3.9753 / (0.9997 - x₅)) - -40.465)
11          4.215e+08  1.395e-01  y = ((x₆ * -1639.6) / (x₄ * (0.97178 - x₅))) - 0.81636
12          3.809e+08  1.013e-01  y = (cube(x₆ * -623.71) / ((x₄ * -1734.5) - x₅)) - 5423.1
───────────────────────────────────────────────────────────────────────────────────────────────────
═════════════════════════════════════════════════════════════════════════════════════════

...

───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           5.978e+08  1.594e+01  y = 2173.3
3           5.868e+08  9.300e-03  y = x₆ * 7047.7
4           5.647e+08  3.830e-02  y = log(x₁₇) * -944.42
5           4.984e+08  1.249e-01  y = -12506 / (x₆ - x₄)
6           4.044e+08  2.091e-01  y = square(x₆ * (368.37 / x₄))
7           3.310e+08  2.004e-01  y = 1497.9 / ((x₆ - -0.31487) - x₄)
8           1.351e+08  8.959e-01  y = square((x₆ * x₅) * (592.73 / x₄))
9           1.045e+08  2.565e-01  y = cube(x₆ * x₅) * square(648.09 / x₄)
10          9.780e+07  6.665e-02  y = (cube(x₅) * 745.25) / ((x₆ - -0.30346) - x₄)
12          9.773e+07  3.673e-04  y = (1983.4 / ((x₆ - -0.30922) - (x₄ + x₁₇))) * cube(x₅)
13          9.132e+07  6.789e-02  y = ((square(x₅) * -1866) / ((x₆ - -0.46234) - x₄)) * log(...
                                      x₁₇)
15          7.388e+07  1.059e-01  y = ((square(x₅) * -

[ Info: Started!



Expressions evaluated per second: 1.420e+04
Progress: 258 / 3100 total iterations (8.323%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           5.269e+08  1.594e+01  y = 1940.7
3           5.261e+08  7.539e-04  y = 2294.5 - x₁₃
4           5.252e+08  1.850e-03  y = cube(x₃ + 10.958)
5           5.164e+08  1.687e-02  y = exp(exp(x₆) + 6.2981)
6           5.010e+08  3.019e-02  y = (1.5194 / sqrt(x₁₇)) + x₈
8           4.895e+08  1.159e-02  y = sqrt(exp(x₁₇)) + (x₈ / sqrt(x₁₇))
11          3.315e+08  1.299e-01  y = ((0.38736 - x₅) * (x₁₂ / x₁₇)) * (x₉ * -1313.6)
13          3.309e+08  8.844e-04  y = (((0.41356 - x₅) * (x₁₂ / x₁₇)) * (x₉ * 51.227)) * -27...
                                      .239
───────────────────────────────────────────────────────────────────────────────────────────

[ Info: Final population:
[ Info: Results saved to:



Expressions evaluated per second: 2.170e+04
Progress: 2752 / 3100 total iterations (88.774%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           5.269e+08  1.594e+01  y = 1942.3
3           5.010e+08  2.521e-02  y = 0.00015195 / x₁₇
4           4.895e+08  2.320e-02  y = x₈ / sqrt(x₁₇)
5           4.475e+08  8.988e-02  y = x₅ * (0.00042106 / x₁₇)
6           2.355e+08  6.420e-01  y = square(x₅ * (x₁₂ / x₁₇))
7           1.530e+08  4.314e-01  y = square(cube(x₅) * (x₁₂ / x₁₇))
8           1.028e+08  3.975e-01  y = square((x₅ * x₁₂) * (x₆ / x₁₇))
9           1.018e+08  9.789e-03  y = cube(x₅) * square(x₁₂ * (x₆ / x₁₇))
10          9.742e+07  4.397e-02  y = cube(((x₅ * x₁₂) / x₁₇) * (x₆ * 0.13332))
12          9.361e+07  1.992e-02  y = cube(((x₆ - x₁₀) * 0.18467) * ((x₅ * x₁₂) / x₁₇))
15 